## Description: Model Fine-tuning (Unsloth) and Quantization (GGUF)
## Author: "Mostafa Shams"

In [1]:
import os
import torch
import gc

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["WANDB_DISABLED"] = "true"

!pip install -q --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-cache-dir --no-deps xformers trl peft accelerate bitsandbytes
!pip install -q evaluate sacrebleu rouge_score

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU Active: {gpu_name} | VRAM: {vram:.2f} GB")
else:
    raise RuntimeError("No GPU detected. Ensure Accelerator is set to GPU T4 x2.")

!CMAKE_ARGS="-DGGML_CUDA=on" pip install -U --force-reinstall --no-cache-dir llama-cpp-python

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 246.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 200.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 332.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 351.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 272.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 349.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 318.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 209.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 167.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 164.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 1

In [2]:
import json
import pandas as pd
from datasets import Dataset

INPUT_FILE = "/kaggle/input/medical/medicalAR_fixed.jsonl"
formatted_rows = []
skipped_count = 0

print(f"Starting Data Sanitization...")

def format_chatml(messages):
    text = ""
    for msg in messages:
        role = msg.get("role", "").strip()
        content = msg.get("content", "").strip()
        if not role or not content or not isinstance(content, str):
            return None 
        text += f"<|im_start|>{role}\n{content}<|im_end|>\n"
    return text + "<|im_start|>assistant\n"

try:
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            try:
                data = json.loads(line)
                
                if "root" in data and "messages" in data["root"]:
                    messages = data["root"]["messages"]
                elif "messages" in data:
                    messages = data["messages"]
                else:
                    skipped_count += 1
                    continue 

                roles = [m['role'] for m in messages]
                if 'user' in roles and 'assistant' in roles:
                    chat_text = format_chatml(messages)
                    if chat_text:
                        formatted_rows.append({
                            "text": chat_text, 
                            "ground_truth": messages[-1]['content']
                        })
                    else:
                        skipped_count += 1
                else:
                    skipped_count += 1
                    
            except Exception as e:
                print(f"Line {i} Error: {e}")
                skipped_count += 1

    full_dataset = Dataset.from_pandas(pd.DataFrame(formatted_rows))
    dataset_split = full_dataset.train_test_split(test_size=0.1)
    train_dataset = dataset_split["train"]
    test_dataset = dataset_split["test"]

    print(f"Data Loaded Successfully!")
    print(f"Total Valid Rows: {len(full_dataset)}")
    print(f"Skipped/Corrupt:  {skipped_count}")
    print(f"Train Size:       {len(train_dataset)}")
    print(f"Test Size:        {len(test_dataset)}")
    
    print("\nFirst Row Sample:")
    print(train_dataset[0]['text'][:300] + "...\n")

except FileNotFoundError:
    print("ERROR: File not found. Check path!")

 Starting Data Sanitization...
Data Loaded Successfully!
   - Total Valid Rows: 2227
   - Skipped/Corrupt:  0
   - Train Size:       2004
   - Test Size:        223

🔍 DEBUG (First Row Sample):

<|im_start|>system
You are a helpful medical assistant. You provide information in Arabic<|im_end|>
<|im_start|>user
إيه هو فحص وظائف الكبد (Liver Function Tests)؟<|im_end|>
<|im_start|>assistant
هي مجموعة من اختبارات الدم التي تقيس مستويات إنزيمات وبروتينات معينة في الدم ينتجها الكبد. يمكن أن تساعد...



In [3]:
from unsloth import FastLanguageModel

max_seq_length = 1024 
dtype = None 
load_in_4bit = True 

print("Loading Qwen 2.5-3B-Instruct (4-bit)...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none", 
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
)

print("Model Loaded & Adapters Applied.")

Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-11-19 21:07:41.211000: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763586461.600394      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763586461.707755      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[xformers|WARNING]WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.9.0+cu128 with CUDA 1208 (you have 2.6.0+cu124)
    Python  3.10.19 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
Using the `WAN

Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.9.0+cu128 with CUDA 1208 (you have 2.6.0+cu124)
    Python  3.10.19 (you have 3.11.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

Loading Qwen 2.5-3B-Instruct (4-bit)...
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


...

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.11.3 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


 Model Loaded & Adapters Applied.


In [4]:
import evaluate

rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')

def calculate_metrics(preds, refs):
    preds = [p if len(p) > 0 else " " for p in preds]
    r = rouge.compute(predictions=preds, references=refs)
    b = bleu.compute(predictions=preds, references=refs)
    return {"bleu": round(b['bleu'], 4), "rouge1": round(r['rouge1'], 4)}

def inference_loop(model_engine, tokenizer, dataset, limit=200, is_gguf=False):
    preds, refs = [], []
    print(f"Running inference on {limit} samples...")
    
    for i in range(limit):
        row = dataset[i]
        prompt = row['text'].split("<|im_start|>assistant")[0] + "<|im_start|>assistant\n"
        ref = row['ground_truth']
        
        if not is_gguf:
            inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
            outputs = model_engine.generate(**inputs, max_new_tokens=100, use_cache=True, pad_token_id=tokenizer.eos_token_id)
            decoded = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:], skip_special_tokens=True)[0]
        else:
            out = model_engine(prompt, max_tokens=100, stop=["<|im_end|>"], echo=False)
            decoded = out['choices'][0]['text']
            
        preds.append(decoded.strip())
        refs.append(ref.strip())
    return preds, refs

print("Running Baseline Inference...")
FastLanguageModel.for_inference(model)
preds_a, refs_a = inference_loop(model, tokenizer, test_dataset, limit=200)
scores_a = calculate_metrics(preds_a, refs_a)
print(f"Baseline Scores: {scores_a}")


Phase A: Running Baseline Inference...
running inference on 200 samples...
Phase A Scores: {'bleu': 0.029, 'rouge1': 0.058}


In [5]:
from trl import SFTTrainer
from transformers import TrainingArguments

print("Starting Full Production Training...")
print(f"Training Set Size: {len(train_dataset)} examples")
print(f"Target Epochs: 3")

model.train() 

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 8,  
        gradient_accumulation_steps = 4,  
        learning_rate = 2e-4,             
        num_train_epochs = 3,             
        max_steps = -1,                   
        warmup_ratio = 0.05,              
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        optim = "adamw_8bit",             
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        logging_steps = 25,               
        save_strategy = "epoch",          
        report_to = "none",               
    ),
)

total_steps = (len(train_dataset) // (4 * 4)) * 3
print(f"Total Estimated Steps: ~{total_steps}")

trainer_stats = trainer.train()
print("Full Training Complete.")


 Starting Full Production Training...
   - Training Set Size: 2004 examples
   - Target Epochs: 3


Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/2004 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


   - Total Estimated Steps: ~375


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,004 | Num Epochs = 3 | Total steps = 189
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 29,933,568 of 3,115,872,256 (0.96% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
25,1.733800
50,1.041900
75,0.866400
100,0.710800
125,0.664600
150,0.523100
175,0.527600


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


 Full Training Complete.


In [6]:
print("Running Fine-Tuned Inference...")

FastLanguageModel.for_inference(model)
preds_b, refs_b = inference_loop(model, tokenizer, test_dataset, limit=200)
scores_b = calculate_metrics(preds_b, refs_b)

print(f"Fine-Tuned Scores: {scores_b}")
print(f"Improvement (BLEU): {scores_b['bleu'] - scores_a['bleu']:.4f}")


 Phase B: Running Fine-Tuned Inference...
running inference on 200 samples...
 Phase B Scores: {'bleu': 0.3296, 'rouge1': 0.1301}
 Improvement (BLEU): 0.3006


In [7]:
print("Converting to GGUF (q5_k_m)...")
save_path = "model_q5_k_m.gguf"

try:
    model.save_pretrained_gguf("model_gguf", tokenizer, quantization_method = "q5_k_m")
    print("GGUF Saved.")
    
    print("Cleaning Memory...")
    del model
    del tokenizer
    del trainer
    gc.collect()
    torch.cuda.empty_cache()
    print(f"VRAM after cleanup: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
    
except Exception as e:
    print(f"Quantization Failed: {e}")


 Converting to GGUF (q5_k_m)...
Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/757 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 2/2 [00:22<00:00, 11.18s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [00:44<00:00, 22.25s/it]


Unsloth: Merge process complete. Saved to `/kaggle/working/model_gguf`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF f16 might take 3 minutes.
\        /    [2] Converting GGUF f16 to ['q5_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: Updating system package directories
Unsloth: All required system packages already installed!
Unsloth: Install llama.cpp and building - please wait 1 to 3 minutes
Unsloth: Cloning llama.cpp repository
Unsloth: Install GGUF and other packages
Unsloth: Successfully installed llama.cpp!
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into f16 GGUF format.
This might take 3 minutes...
Unsloth: Initial conversion completed! Files: ['qwen2.5-3b-instruct.F16.gguf']
Un

In [8]:
!pip install scikit-build-core cmake
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121

  Using cached scikit_build_core-0.11.6-py3-none-any.whl.metadata (18 kB)
Using cached scikit_build_core-0.11.6-py3-none-any.whl (185 kB)
Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 551.3/551.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.7 MB/s eta 0:00:00


In [9]:
from llama_cpp import Llama

print("Verifying GGUF on Edge Runtime...")
full_path = "/kaggle/working/qwen2.5-3b-instruct.Q5_K_M.gguf"

try:
    edge_model = Llama(
        model_path=full_path,
        n_gpu_layers=-1, 
        verbose=False,
        n_ctx=2048
    )
    
    preds_c, refs_c = inference_loop(edge_model, None, test_dataset, limit=200, is_gguf=True)
    scores_c = calculate_metrics(preds_c, refs_c)
    
    print(f"Verification Scores: {scores_c}")
    print(f"Quantization Loss (BLEU): {scores_b['bleu'] - scores_c['bleu']:.4f}")

except Exception as e:
    print(f"Verification Failed: {e}")

print("Pipeline Complete. Ready for download.")


📱 Phase C: Verifying GGUF on Edge Runtime...


llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


running inference on 200 samples...
 Phase C Scores: {'bleu': 0.1606, 'rouge1': 0.0938}
 Quantization Loss (BLEU): 0.1690

 PIPELINE COMPLETE. Download 'model_gguf/model_q5_k_m.gguf' for your portfolio.


In [10]:
import torch
import gc
import time
import os
from unsloth import FastLanguageModel
from llama_cpp import Llama
import evaluate

rouge = evaluate.load('rouge')
bleu = evaluate.load('bleu')

def calculate_metrics(preds, refs):
    preds = [p if len(p) > 0 else " " for p in preds]
    r = rouge.compute(predictions=preds, references=refs)
    b = bleu.compute(predictions=preds, references=refs)
    return {"bleu": round(b['bleu'], 4), "rouge1": round(r['rouge1'], 4)}

def get_vram_gb():
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated(0) / 1024**3
    return 0

def benchmark_model(model, tokenizer, prompts, refs, is_gguf=False):
    times = []
    token_counts = []
    preds = []
    
    print(f"Running Inference on {len(prompts)} samples...")
    
    for prompt in prompts:
        start = time.time()
        if not is_gguf:
            inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
            gen_out = model.generate(
                **inputs, 
                max_new_tokens=128, 
                use_cache=True, 
                pad_token_id=tokenizer.eos_token_id
            )
            end = time.time()
            generated_ids = gen_out[:, inputs.input_ids.shape[1]:]
            num_tokens = generated_ids.shape[1]
            output_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        else:
            res = model(
                prompt, 
                max_tokens=128, 
                stop=["<|im_end|>"], 
                echo=False
            )
            end = time.time()
            num_tokens = res['usage']['completion_tokens']
            output_text = res['choices'][0]['text']
            
        times.append(end - start)
        token_counts.append(num_tokens)
        preds.append(output_text.strip())
        
    avg_speed = sum(token_counts) / sum(times)
    metrics = calculate_metrics(preds, refs)
    return avg_speed, metrics

try:
    subset = test_dataset.select(range(200)) 
    prompts = [row['text'].split("<|im_start|>assistant")[0] + "<|im_start|>assistant\n" for row in subset]
    refs = [row['ground_truth'] for row in subset]
except:
    print("Test dataset missing. Using dummy data.")
    prompts = ["<|im_start|>user\nHello<|im_end|>\n<|im_start|>assistant\n"] * 5
    refs = ["Hi"] * 5

print("Loading Full Model (FP16)...")
gc.collect()
torch.cuda.empty_cache()
start_mem = get_vram_gb()

model_fp16, tokenizer_fp16 = FastLanguageModel.from_pretrained(
    model_name = "model_gguf",  
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = False,
)
FastLanguageModel.for_inference(model_fp16)

fp16_mem = get_vram_gb() - start_mem
print(f"FP16 VRAM Usage: {fp16_mem:.2f} GB")

fp16_speed, fp16_acc = benchmark_model(model_fp16, tokenizer_fp16, prompts, refs, is_gguf=False)
print(f"FP16 Speed: {fp16_speed:.2f} tokens/sec")
print(f"FP16 Accuracy: {fp16_acc}")

del model_fp16, tokenizer_fp16
gc.collect()
torch.cuda.empty_cache()

print("Loading Quantized Model (GGUF)...")
gguf_path = "qwen2.5-3b-instruct.Q5_K_M.gguf" 

if not os.path.exists(gguf_path):
    if os.path.exists(f"/kaggle/working/{gguf_path}"):
         gguf_path = f"/kaggle/working/{gguf_path}"
    else:
        print(f"Error: GGUF file not found at {gguf_path}")

model_gguf = Llama(
    model_path=gguf_path,
    n_gpu_layers=-1,
    verbose=False,
    n_ctx=2048
)

gguf_size = os.path.getsize(gguf_path) / 1024**3
gguf_mem = gguf_size + 0.5 
print(f"GGUF VRAM Usage: ~{gguf_mem:.2f} GB (Est)")

gguf_speed, gguf_acc = benchmark_model(model_gguf, None, prompts, refs, is_gguf=True)
print(f"GGUF Speed: {gguf_speed:.2f} tokens/sec")
print(f"GGUF Accuracy: {gguf_acc}")

print("\n" + "="*60)
print("FINAL BENCHMARK: Full Precision vs Quantization")
print("="*60)
print(f"{'Metric':<20} | {'FP16 (Full)':<15} | {'GGUF (Q5)':<15} | {'Impact':<15}")
print("-" * 70)

mem_delta = ((fp16_mem - gguf_mem) / fp16_mem) * 100
print(f"{'Memory (VRAM)':<20} | {fp16_mem:.4f} GB        | {gguf_mem:.4f} GB        | {mem_delta:.2f}% Smaller")

speed_delta = ((gguf_speed - fp16_speed) / fp16_speed) * 100
print(f"{'Inference Speed':<20} | {fp16_speed:.5f} t/s        | {gguf_speed:.5f} t/s        | {speed_delta:+.2f}% Change")

acc_delta = gguf_acc['bleu'] - fp16_acc['bleu']
print(f"{'Accuracy (BLEU)':<20} | {fp16_acc['bleu']:<15} | {gguf_acc['bleu']:<15} | {acc_delta:+.5f} Delta")
print("-" * 70)


🔹 Phase 1: Loading Full Model (FP16) from 'model_gguf' folder...
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

    FP16 VRAM Usage: 5.77 GB
    Running Inference on 200 samples...
    FP16 Speed: 24.30 tokens/sec
    FP16 Accuracy: {'bleu': 0.2978, 'rouge1': 0.1447}

🔹 Phase 2: Loading Quantized Model (GGUF Q5_K_M)...


llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


    GGUF VRAM Usage: ~2.57 GB (Est)
    Running Inference on 200 samples...
    GGUF Speed: 54.49 tokens/sec
    GGUF Accuracy: {'bleu': 0.1993, 'rouge1': 0.0938}

🚀 FINAL BENCHMARK: Full Precision vs Quantization
Metric               | FP16 (Full)     | GGUF (Q5)       | Impact         
----------------------------------------------------------------------
Memory (VRAM)        | 5.7655 GB        | 2.5720 GB        | 55.39% Smaller
Inference Speed      | 24.30384 t/s        | 54.49194 t/s        | +124.21% Change
Accuracy (BLEU)      | 0.2978          | 0.1993          | -0.09850 Delta
----------------------------------------------------------------------
